In [42]:
import pandas as pd

json_dev = pd.read_json("data/dev-v2.0.json")
json_train = pd.read_json("data/train-v2.0.json")

In [43]:
def data_extraction(json_data):
    extracted_data = []

    # Iterate through each item in the 'data' column
    for item in json_data['data']:
        title = item['title']  # Access 'title' within the item
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                # Assuming we take the first answer provided for each question
                if qa['answers']:
                    answer_text = qa['answers'][0]['text']
                    answer_start = qa['answers'][0]['answer_start']
                    answer_end = answer_start + len(answer_text)
                else:
                    # Handle cases where there might be no answers (e.g., for is_impossible=True)
                    answer_text = None
                    answer_start = None
                    answer_end = None

                extracted_data.append({
                    'title': title,
                    'context': context,
                    'question': question,
                    'answer': answer_text,
                    'answer_start': answer_start,
                    'answer_end': answer_end
                })

    return pd.DataFrame(extracted_data)

In [44]:
dev_columns = data_extraction(json_dev)
train_columns = data_extraction(json_train)